In [ ]:
!pip install fairlearn aif360

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install 'aif360[inFairness]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Core
import numpy as np
import pandas as pd

# Scikit‑learn preprocessing / modelling
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Fairness libraries
from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric


/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [ ]:
# ──────────────────────────────────────────────
# 0.  Imports
# ──────────────────────────────────────────────
import pathlib, urllib.request, os, pprint

# ──────────────────────────────────────────────
# 1.  Create the directory
#     /content/aif360_data/raw/adult
# ──────────────────────────────────────────────
data_root = pathlib.Path("/content/aif360_data/raw/adult")
data_root.mkdir(parents=True, exist_ok=True)   # parents=True builds any missing folders

# ──────────────────────────────────────────────
# 2.  Download the three files if they do NOT
#     already exist (keeps reruns fast)
# ──────────────────────────────────────────────
urls = {
    "adult.data" : "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "adult.test" : "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    "adult.names": "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",
}

for fname, url in urls.items():
    dest = data_root / fname
    if dest.exists():
        print(f"✓ {fname} already present")
    else:
        print(f"⬇ Downloading {fname}")
        urllib.request.urlretrieve(url, dest)

# ──────────────────────────────────────────────
# 3.  Quick sanity check: show what’s there
# ──────────────────────────────────────────────
print("\nFiles in", data_root, ":")
pprint.pp([p.name for p in data_root.iterdir()])

# ──────────────────────────────────────────────
# 4.  Tell AIF360 where to find the data
# ──────────────────────────────────────────────
os.environ["AIF360_DATA_DIR"] = str(data_root.parent)   # → /content/aif360_data/raw


✓ adult.data already present
✓ adult.test already present
✓ adult.names already present

Files in /content/aif360_data/raw/adult :
['adult.names', 'adult.data', 'adult.test']


In [ ]:
import importlib.util, pathlib, shutil, pprint

# ------------------------------------------------------------------
# Where is aif360 installed?
# ------------------------------------------------------------------
pkg_dir = pathlib.Path(importlib.util.find_spec("aif360").origin).parent
target_dir = pkg_dir / "data" / "raw" / "adult"
target_dir.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# Source files (you created them earlier)
# ------------------------------------------------------------------
src_dir = pathlib.Path("/content/aif360_data/raw/adult")
required = ["adult.data", "adult.test", "adult.names"]

for fname in required:
    src = src_dir / fname
    dst = target_dir / fname
    shutil.copy2(src, dst)
    print(f"✓ Copied {fname}  →  {dst}")

print("\nFiles now present at:")
pprint.pp([p.name for p in target_dir.iterdir()])


✓ Copied adult.data  →  /usr/local/lib/python3.11/dist-packages/aif360/data/raw/adult/adult.data
✓ Copied adult.test  →  /usr/local/lib/python3.11/dist-packages/aif360/data/raw/adult/adult.test
✓ Copied adult.names  →  /usr/local/lib/python3.11/dist-packages/aif360/data/raw/adult/adult.names

Files now present at:
['adult.names', 'README.md', 'adult.data', 'adult.test']


In [ ]:
from aif360.datasets import AdultDataset

adult_raw = AdultDataset()               # ← should load with no IOError now
print("Loaded AdultDataset — features shape:", adult_raw.features.shape)


Loaded AdultDataset — features shape: (45222, 98)


In [ ]:
adult_df, _ = adult_raw.convert_to_dataframe()
display(adult_df.head())
print("Missing‑value counts:\n", adult_df.isna().sum())


,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year
0,25.0,7.0,0.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,9.0,1.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,12.0,1.0,1.0,0.0,0.0,40.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,44.0,10.0,0.0,1.0,7688.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,34.0,6.0,1.0,1.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Missing‑value counts:
 age                               0
education-num                     0
race                              0
sex                               0
capital-gain                      0
                                 ..
native-country=Trinadad&Tobago    0
native-country=United-States      0
native-country=Vietnam            0
native-country=Yugoslavia         0
income-per-year                   0
Length: 99, dtype: int64


In [ ]:
label_col       = "income-per-year"
protected_attr  = "sex"

X = adult_df.drop(columns=[label_col])
y = adult_df[label_col].eq(">50K").astype(int)   # 1 => high income

numeric_cols     = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(include="object").columns
print(f"{len(numeric_cols)} numeric | {len(categorical_cols)} categorical features")


98 numeric | 0 categorical features


In [ ]:
from sklearn.pipeline    import Pipeline
from sklearn.compose     import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute      import SimpleImputer
from sklearn.linear_model import LogisticRegression

# ----- Pipeline WITH imputation -----
impute_pre = ColumnTransformer([
    ('num', Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc",  StandardScaler())]), numeric_cols),
    ('cat', Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh",  OneHotEncoder(handle_unknown="ignore"))]), categorical_cols)
])
impute_clf = Pipeline([("pre", impute_pre),
                       ("clf", LogisticRegression(max_iter=500))])

# ----- Pipeline WITHOUT imputation -----
raw_pre = ColumnTransformer([
    ('num', Pipeline([("sc", StandardScaler())]), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])
raw_clf = Pipeline([("pre", raw_pre),
                    ("clf", LogisticRegression(max_iter=500))])


In [ ]:
y = adult_df[label_col].eq(">50K").astype(int)


In [ ]:
# 1.  Inspect unique values first
print(adult_df[label_col].unique()[:10])

# 2.  If they are already 0 / 1, just cast to int:
y = adult_df[label_col].astype(int)

#    Otherwise normalise the strings:
# y = adult_df[label_col].str.strip().str.replace('.', '', regex=False).eq(">50K").astype(int)

# 3.  Sanity‑check the class balance
print("Class counts:\n", y.value_counts())


[0. 1.]
Class counts:
 income-per-year
0    34014
1    11208
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=2025
)

raw_clf.fit(X_train, y_train)
impute_clf.fit(X_train, y_train)
print("✅ both models trained")


✅ both models trained


In [ ]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import selection_rate, MetricFrame

def fairlearn_report(model, title):
    preds = model.predict(X_test)

    mf = MetricFrame(
        metrics = {
            "accuracy"      : accuracy_score,
            "selection_rate": selection_rate          # ⬅ needs no extra args
        },
        y_true = y_test,
        y_pred = preds,
        sensitive_features = X_test[protected_attr]
    )

    # overall
    print(f"\n🔹 {title}")
    display(mf.overall.to_frame("overall").T)

    # by‑group
    display(mf.by_group.T)

    # disparity metrics (computed once, outside MetricFrame):
    from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
    dp  = demographic_parity_difference(y_test, preds,
                                        sensitive_features=X_test[protected_attr])
    eo  = equalized_odds_difference(y_test, preds,
                                    sensitive_features=X_test[protected_attr])
    print(f"Demographic‑parity diff  : {dp:+.3f}")
    print(f"Equalized‑odds diff      : {eo:+.3f}")

fairlearn_report(raw_clf,    "NO IMPUTATION")
fairlearn_report(impute_clf, "AFTER IMPUTATION")



🔹 NO IMPUTATION


,accuracy,selection_rate
overall,0.849193,0.197096


sex,0.0,1.0
accuracy,0.922937,0.813809
selection_rate,0.076381,0.255017


Demographic‑parity diff  : +0.179
Equalized‑odds diff      : +0.114

🔹 AFTER IMPUTATION


,accuracy,selection_rate
overall,0.849193,0.197096


sex,0.0,1.0
accuracy,0.922937,0.813809
selection_rate,0.076381,0.255017


Demographic‑parity diff  : +0.179
Equalized‑odds diff      : +0.114
